# Loading and Storing Data : 
#### - The dataset is appropriately loaded and stored into corresponding variables.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Check List : MLP 11 things 

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, RobustScaler,OrdinalEncoder
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, MultiLabelBinarizer

In [4]:
Movies_Data=pd.read_csv('C:/Users/Sameem/Desktop/MLPProject/movies.csv')
Train_Data=pd.read_csv('C:/Users/Sameem/Desktop/MLPProject/train (1).csv')

In [5]:
Test_Data=pd.read_csv('C:/Users/Sameem/Desktop/MLPProject/test.csv')
#Sample_Data=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv')

In [6]:
print(Test_Data.shape)


(55315, 4)


In [7]:
MDC = Movies_Data.copy()

In [8]:
MDC.head()

,movieid,title,audienceScore,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,boxOffice,distributor,soundType
0,han_solo_heroic_myth_wonder_woman,Han Solo Heroic Myth Wonder Woman,50.0,NaN,NaN,NaN,2018-08-25,75.0,"Comedy, Horror, Sci-fi",English,Claude Nicolet,NaN,NaN,NaN
1,voyage_tyler_durden_han_solo_stardust,Voyage Tyler Durden Han Solo Stardust,NaN,NaN,NaN,NaN,2020-02-11,114.0,Drama,English,Nathan Haack,NaN,NaN,NaN
2,norman_bates_ferris_bueller_hermione_granger_v...,Norman Bates Ferris Bueller Hermione Granger V...,43.0,NaN,NaN,NaN,NaN,120.0,Drama,Korean,Theresa Smith,NaN,NaN,NaN
3,elegant_hermione_granger,Elegant Hermione Granger,60.0,NaN,NaN,NaN,2020-10-23,90.0,"Action, Mystery & thriller",English,Donald Lewis,NaN,NaN,NaN
4,adventure_rocky_balboa,Adventure Rocky Balboa,70.0,NaN,NaN,NaN,2017-03-27,80.0,"Fantasy, Adventure, Animation",English,Howard Simms,NaN,NaN,NaN


In [9]:
Movies_Data.shape==MDC.shape

True

# Exploratory data analysis
#### - Visualizing key statistics and relationships in the data
#### - Correctly identify the feature types.

In [10]:
MDC.nunique()

movieid                 126404
title                   126404
audienceScore              101
rating                      10
ratingContents            8353
releaseDateTheaters      12062
releaseDateStreaming      4726
runtimeMinutes             324
genre                     2912
originalLanguage           112
director                 62208
boxOffice                 4863
distributor               3694
soundType                  551
dtype: int64

In [11]:
MDC.shape

(143258, 14)

* Removing Duplicates

In [12]:
MDC.duplicated(subset="movieid").sum()

16854

In [13]:
''' We need to drop duplicates , but we need to drop that duplicates that has the most number of null values ensuring that the data with more non-null information is retained.So we do it like below, since the drop_duplicates method drops all occurences except the first or last depending on how we call it.'''

' We need to drop duplicates , but we need to drop that duplicates that has the most number of null values ensuring that the data with more non-null information is retained.So we do it like below, since the drop_duplicates method drops all occurences except the first or last depending on how we call it.'

In [14]:
null_counts = MDC.isnull().sum(axis=1)
null_counts

0         6
1         7
2         7
3         6
4         6
         ..
143253    5
143254    8
143255    7
143256    7
143257    8
Length: 143258, dtype: int64

In [15]:
MDC = MDC.assign(null_counts=null_counts).sort_values(by='null_counts')


In [16]:
MDC=MDC.drop_duplicates(subset=['movieid'],keep='first')
MDC.shape

(126404, 15)

In [16]:
MDC.drop('null_counts',axis=1,inplace=True)

In [17]:
# MDC.describe()

In [18]:
TDC=Train_Data.copy()

In [19]:
# TDC.nunique()

In [20]:
# MDC.nunique()

In [21]:
# null_rows=MDC.isnull().sum()
# total_rows=len(MDC)
# null_percentages=(null_rows/total_rows)*100
# print(null_percentages)

* As we can see here `rating` , `ratingContents`, `releaseDateTheaters`,`boxOffice`,`distributor` &`soundType` have large percentage (>75%) of null values so they won't help much in EDA , thus we will drop them

In [22]:
columns_to_drop = ['rating', 'ratingContents', 'releaseDateTheaters', 'boxOffice', 'distributor', 'soundType']
MDC.drop(columns=columns_to_drop, axis=1,inplace=True)

# Display the DataFrame after dropping the columns
MDC.head(2)

,movieid,title,audienceScore,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director
122446,enthralling_riddle_whisper,Enthralling Riddle Whisper,45.0,2014-01-28,103.0,"Comedy, Drama",English,Billie Aguirre
66214,e.t._dragon_neo_phantom,E.T. Dragon Neo Phantom,53.0,2007-09-25,96.0,"Sci-fi, Action",English,Timothy Baim


In [23]:
# null_rows=MDC.isnull().sum()
# total_rows=len(MDC)
# null_percentages=(null_rows/total_rows)*100
# print(null_percentages)

* Removing invalid dates and extracting year from `releaseDateStreaming` 

In [24]:
MDC['releaseDateStreaming'] = pd.to_datetime(MDC['releaseDateStreaming'], errors='coerce')
MDC['releaseDateStreaming'] = pd.to_datetime(MDC['releaseDateStreaming'])
MDC['releaseDateStreamingDay']= MDC['releaseDateStreaming'].dt.day
MDC['releaseDateStreamingMonth']= MDC['releaseDateStreaming'].dt.month
MDC['releaseDateStreaming'] = MDC['releaseDateStreaming'].dt.year
insert_index = 3

new_column_name = MDC.columns[-1]

MDC = pd.concat([MDC.iloc[:, :insert_index], MDC[new_column_name], MDC.iloc[:, insert_index:-1]], axis=1)

insert_index = 3

new_column_name = MDC.columns[-1]

MDC = pd.concat([MDC.iloc[:, :insert_index], MDC[new_column_name], MDC.iloc[:, insert_index:-1]], axis=1)

In [25]:
MDC.columns

Index(['movieid', 'title', 'audienceScore', 'releaseDateStreamingDay',
       'releaseDateStreamingMonth', 'releaseDateStreaming', 'runtimeMinutes',
       'genre', 'originalLanguage', 'director'],
      dtype='object')

In [26]:
MDC.head(2)

,movieid,title,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director
122446,enthralling_riddle_whisper,Enthralling Riddle Whisper,45.0,28.0,1.0,2014.0,103.0,"Comedy, Drama",English,Billie Aguirre
66214,e.t._dragon_neo_phantom,E.T. Dragon Neo Phantom,53.0,25.0,9.0,2007.0,96.0,"Sci-fi, Action",English,Timothy Baim


* Since `movieid` and `title` are similar columns , we will drop one of them

In [27]:
MDC.drop(columns=['title'],axis=1,inplace=True)


In [28]:
MDC.head(1)

,movieid,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director
122446,enthralling_riddle_whisper,45.0,28.0,1.0,2014.0,103.0,"Comedy, Drama",English,Billie Aguirre


In [29]:
MDC['genre'].value_counts()
Train_Data['sentiment'].value_counts([0])

POSITIVE    0.668238
NEGATIVE    0.331762
Name: sentiment, dtype: float64

In [30]:
MDC['genre'].value_counts()

Drama                                                  23806
Documentary                                            14098
Comedy                                                 10521
Mystery & thriller                                      5716
Comedy, Drama                                           4883
                                                       ...  
Kids & family, Mystery & thriller, Adventure, Drama        1
Action, Adventure, Drama, Sci-fi, Western                  1
Horror, Romance, Lgbtq+                                    1
Musical, Drama, Adventure                                  1
Adventure, Music                                           1
Name: genre, Length: 2773, dtype: int64

In [31]:
MDC.columns 

Index(['movieid', 'audienceScore', 'releaseDateStreamingDay',
       'releaseDateStreamingMonth', 'releaseDateStreaming', 'runtimeMinutes',
       'genre', 'originalLanguage', 'director'],
      dtype='object')

In [32]:
MDC['genre'].unique()
MDC['genre1'] = MDC['genre'].str.split(', ')
MDC.head(2)

,movieid,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,genre1
122446,enthralling_riddle_whisper,45.0,28.0,1.0,2014.0,103.0,"Comedy, Drama",English,Billie Aguirre,"[Comedy, Drama]"
66214,e.t._dragon_neo_phantom,53.0,25.0,9.0,2007.0,96.0,"Sci-fi, Action",English,Timothy Baim,"[Sci-fi, Action]"


In [33]:
bigger_list=[]
MDC['genre1'].apply(lambda sublist: bigger_list.extend(sublist) if isinstance(sublist, list) else None)
series = pd.Series(bigger_list)


genre_counts = series.value_counts()
top_3_genres = genre_counts.head(3)
print(top_3_genres)

Drama          48146
Comedy         27397
Documentary    17645
dtype: int64


In [34]:
MDC.drop('genre1',axis=1, inplace=True )

In [35]:
MDC['genre'].fillna('Drama, Comedy, Documentary',inplace=True)

In [36]:
MDC.isnull().sum()

movieid                          0
audienceScore                60619
releaseDateStreamingDay      55103
releaseDateStreamingMonth    55103
releaseDateStreaming         55103
runtimeMinutes               11988
genre                            0
originalLanguage             11883
director                         0
dtype: int64

In [37]:
MDC.shape

(126404, 9)

## Imputing 

In [38]:
columns_to_impute = ['audienceScore', 'releaseDateStreamingDay', 'releaseDateStreamingMonth',
                     'releaseDateStreaming', 'runtimeMinutes']

# Create a SimpleImputer with strategy='mean'
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to the specified columns
MDC[columns_to_impute] = imputer.fit_transform(MDC[columns_to_impute])
MDC[columns_to_impute] =(MDC[columns_to_impute]).round().astype(int)

In [39]:
imputer = SimpleImputer(strategy='most_frequent')

# Apply the imputer to the 'originalLanguage' column
MDC['originalLanguage'] = imputer.fit_transform(MDC[['originalLanguage']])


In [40]:
MDC.isnull().sum()

movieid                      0
audienceScore                0
releaseDateStreamingDay      0
releaseDateStreamingMonth    0
releaseDateStreaming         0
runtimeMinutes               0
genre                        0
originalLanguage             0
director                     0
dtype: int64

In [90]:
MDC.shape

(126404, 9)

In [41]:
tDC= Test_Data.copy()

In [42]:
merged_Train = pd.merge(TDC,MDC, on='movieid', how='left')

In [43]:
merged_Test = pd.merge( tDC,MDC, on='movieid', how='left')

In [44]:
merged_Train.shape

(162758, 13)

In [45]:
Train_Data.columns

Index(['movieid', 'reviewerName', 'isFrequentReviewer', 'reviewText',
       'sentiment'],
      dtype='object')

In [46]:
merged_Train.isnull().sum()

movieid                         0
reviewerName                    0
isFrequentReviewer              0
reviewText                   6447
sentiment                       0
audienceScore                   0
releaseDateStreamingDay         0
releaseDateStreamingMonth       0
releaseDateStreaming            0
runtimeMinutes                  0
genre                           0
originalLanguage                0
director                        0
dtype: int64

In [47]:
merged_Train['sentiment'].value_counts()

POSITIVE    108761
NEGATIVE     53997
Name: sentiment, dtype: int64

In [48]:
merged_Train.isnull().sum()

movieid                         0
reviewerName                    0
isFrequentReviewer              0
reviewText                   6447
sentiment                       0
audienceScore                   0
releaseDateStreamingDay         0
releaseDateStreamingMonth       0
releaseDateStreaming            0
runtimeMinutes                  0
genre                           0
originalLanguage                0
director                        0
dtype: int64

In [49]:
filtered_rows_pos = merged_Train[(merged_Train['reviewText'].isnull()) & (merged_Train['sentiment'] == 'POSITIVE')]
print(filtered_rows_pos.shape)
filtered_rows_neg = merged_Train[(merged_Train['reviewText'].isnull()) & (merged_Train['sentiment'] == 'NEGATIVE')]
print(filtered_rows_neg.shape)

(4042, 13)
(2405, 13)


In [50]:
replacement_string = 'good masterpiece outstanding fantastic beautiful gorgeous marvelous beauty fabulous pleasing wonderful great'
merged_Train.loc[filtered_rows_pos.index, 'reviewText'] = replacement_string

In [51]:
merged_Test.fillna(replacement_string, inplace=True)

In [52]:
merged_Train.isnull().sum()

movieid                         0
reviewerName                    0
isFrequentReviewer              0
reviewText                   2405
sentiment                       0
audienceScore                   0
releaseDateStreamingDay         0
releaseDateStreamingMonth       0
releaseDateStreaming            0
runtimeMinutes                  0
genre                           0
originalLanguage                0
director                        0
dtype: int64

In [89]:
merged_Train.shape

(162758, 66)

In [53]:
replacement_string2 = 'bad worst worse cheap poor diminish overrated nonsense waste retarded iffy upset upsetting rubbish '
merged_Train.loc[filtered_rows_neg.index, 'reviewText'] = replacement_string2

In [54]:
merged_Train.isnull().sum()

movieid                      0
reviewerName                 0
isFrequentReviewer           0
reviewText                   0
sentiment                    0
audienceScore                0
releaseDateStreamingDay      0
releaseDateStreamingMonth    0
releaseDateStreaming         0
runtimeMinutes               0
genre                        0
originalLanguage             0
director                     0
dtype: int64

* Custom function for `genre`

In [55]:
merged_Train['genre'].isnull().sum()

0

In [56]:
genre_array_Train = merged_Train['genre'].values
genre_array_Test = merged_Test['genre'].values

In [57]:
genre_array_Train

array(['Comedy, Animation, Adventure, Fantasy',
       'Holiday, Romance, Comedy, Lgbtq+', 'Drama', ..., 'Comedy, Drama',
       'Kids & family, Holiday, Comedy', 'Horror, Mystery & thriller'],
      dtype=object)

In [58]:
merged_Train.shape
merged_Train.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE,65,28,10,2022,105,"Comedy, Animation, Adventure, Fantasy",English,Bennie Basso
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE,47,18,5,2010,119,"Holiday, Romance, Comedy, Lgbtq+",English,Hubert Dixon
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE,80,12,2,2019,130,Drama,English,Bobby Oleary
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE,47,20,1,2015,88,"Action, Mystery & thriller, Sci-fi",English,Gail Pugh
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE,97,8,6,2004,131,"Crime, Drama",Portuguese (Brazil),John Trent


In [59]:
#for Train 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer

merged_Train['genre'] = merged_Train['genre'].str.lower().str.split(',')
count_vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x)

count_vectorizer.fit(merged_Train['genre'])


genre_matrix1 = count_vectorizer.transform(merged_Train['genre'])
genre_df1 = pd.DataFrame(genre_matrix1.toarray(), columns=count_vectorizer.get_feature_names_out())

merged_Train = pd.merge(merged_Train, genre_df1, left_index=True, right_index=True)
merged_Train.drop(columns='genre', inplace=True)

merged_Train.head()

c:\Users\Sameem\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,...,horror,kids & family,music,musical,mystery & thriller,romance,sci-fi,stand-up,war,western
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE,65,28,10,2022,105,...,0,0,0,0,0,0,0,0,0,0
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE,47,18,5,2010,119,...,0,0,0,0,0,0,0,0,0,0
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE,80,12,2,2019,130,...,0,0,0,0,0,0,0,0,0,0
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE,47,20,1,2015,88,...,0,0,0,0,0,0,0,0,0,0
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE,97,8,6,2004,131,...,0,0,0,0,0,0,0,0,0,0


In [60]:
#for Test
merged_Test['genre'] = merged_Test['genre'].str.lower().str.split(',')
count_vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x)

count_vectorizer.fit(merged_Test['genre'])


genre_matrix = count_vectorizer.transform(merged_Test['genre'])
genre_df = pd.DataFrame(genre_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

merged_Test = pd.merge(merged_Test, genre_df, left_index=True, right_index=True)
merged_Test.drop(columns='genre', inplace=True)

merged_Test.shape

c:\Users\Sameem\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(55315, 65)

* Custom function for `originalLanguage`

* `isFrequentReviewer` bool to int 

In [61]:
merged_Test.rename(columns={'isTopCritic': 'isFrequentReviewer'}, inplace=True)

In [62]:
merged_Train['isFrequentReviewer'] = merged_Train['isFrequentReviewer'].astype(int)

merged_Test['isFrequentReviewer'] = merged_Test['isFrequentReviewer'].astype(int)

In [63]:
merged_Train.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,...,horror,kids & family,music,musical,mystery & thriller,romance,sci-fi,stand-up,war,western
0,marvelous_pirate,Benjamin Henry,0,Henry Selick’s first movie since 2009’s Corali...,POSITIVE,65,28,10,2022,105,...,0,0,0,0,0,0,0,0,0,0
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,0,With a cast that reads like the Vogue Oscar pa...,NEGATIVE,47,18,5,2010,119,...,0,0,0,0,0,0,0,0,0,0
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,1,Creed II does not give us anything but another...,POSITIVE,80,12,2,2019,130,...,0,0,0,0,0,0,0,0,0,0
3,lara_croft_glimmer,Ryan Barrett,0,"I know what you're thinking, but this is no Li...",POSITIVE,47,20,1,2015,88,...,0,0,0,0,0,0,0,0,0,0
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,0,Director Fernando Meirelles tells the story wi...,POSITIVE,97,8,6,2004,131,...,0,0,0,0,0,0,0,0,0,0


* Split data into X and y 

In [64]:
merged_Train.isnull().sum()

movieid               0
reviewerName          0
isFrequentReviewer    0
reviewText            0
sentiment             0
                     ..
romance               0
sci-fi                0
stand-up              0
war                   0
western               0
Length: 66, dtype: int64

In [65]:
Test_Data.nunique()

movieid         16786
reviewerName     4495
isTopCritic         2
reviewText      52686
dtype: int64

In [66]:
merged_Test.rename(columns={'isTopCritic': 'isFrequentReviewer'}, inplace=True)

In [67]:
merged_Test.isnull().sum()

movieid               0
reviewerName          0
isFrequentReviewer    0
reviewText            0
audienceScore         0
                     ..
romance               0
sci-fi                0
stand-up              0
war                   0
western               0
Length: 65, dtype: int64

In [68]:
merged_Train.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,audienceScore,releaseDateStreamingDay,releaseDateStreamingMonth,releaseDateStreaming,runtimeMinutes,...,horror,kids & family,music,musical,mystery & thriller,romance,sci-fi,stand-up,war,western
0,marvelous_pirate,Benjamin Henry,0,Henry Selick’s first movie since 2009’s Corali...,POSITIVE,65,28,10,2022,105,...,0,0,0,0,0,0,0,0,0,0
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,0,With a cast that reads like the Vogue Oscar pa...,NEGATIVE,47,18,5,2010,119,...,0,0,0,0,0,0,0,0,0,0
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,1,Creed II does not give us anything but another...,POSITIVE,80,12,2,2019,130,...,0,0,0,0,0,0,0,0,0,0
3,lara_croft_glimmer,Ryan Barrett,0,"I know what you're thinking, but this is no Li...",POSITIVE,47,20,1,2015,88,...,0,0,0,0,0,0,0,0,0,0
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,0,Director Fernando Meirelles tells the story wi...,POSITIVE,97,8,6,2004,131,...,0,0,0,0,0,0,0,0,0,0


In [69]:
merged_Train.dtypes

movieid               object
reviewerName          object
isFrequentReviewer     int32
reviewText            object
sentiment             object
                       ...  
romance                int64
sci-fi                 int64
stand-up               int64
war                    int64
western                int64
Length: 66, dtype: object

In [70]:
X=merged_Train.drop(columns=['releaseDateStreaming','releaseDateStreamingMonth','releaseDateStreaming','sentiment','originalLanguage','runtimeMinutes'],axis=1)
y=merged_Train['sentiment']

In [88]:
X.shape

(162758, 59)

In [71]:
X.columns

Index(['movieid', 'reviewerName', 'isFrequentReviewer', 'reviewText',
       'audienceScore', 'releaseDateStreamingDay', 'director', ' action',
       ' adventure', ' animation', ' anime', ' biography', ' comedy', ' crime',
       ' documentary', ' drama', ' faith & spirituality', ' fantasy',
       ' foreign', ' gay & lesbian', ' health & wellness', ' history',
       ' holiday', ' horror', ' kids & family', ' lgbtq+', ' music',
       ' musical', ' mystery & thriller', ' nature', ' news', ' romance',
       ' sci-fi', ' short', ' sports', ' sports & fitness', ' stand-up',
       ' variety', ' war', ' western', 'action', 'adventure', 'animation',
       'biography', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
       'history', 'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [72]:
X.drop(['reviewerName', 'director'], axis = 1, inplace = True)

In [73]:
X.columns

Index(['movieid', 'isFrequentReviewer', 'reviewText', 'audienceScore',
       'releaseDateStreamingDay', ' action', ' adventure', ' animation',
       ' anime', ' biography', ' comedy', ' crime', ' documentary', ' drama',
       ' faith & spirituality', ' fantasy', ' foreign', ' gay & lesbian',
       ' health & wellness', ' history', ' holiday', ' horror',
       ' kids & family', ' lgbtq+', ' music', ' musical',
       ' mystery & thriller', ' nature', ' news', ' romance', ' sci-fi',
       ' short', ' sports', ' sports & fitness', ' stand-up', ' variety',
       ' war', ' western', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'history',
       'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [74]:
X.columns

Index(['movieid', 'isFrequentReviewer', 'reviewText', 'audienceScore',
       'releaseDateStreamingDay', ' action', ' adventure', ' animation',
       ' anime', ' biography', ' comedy', ' crime', ' documentary', ' drama',
       ' faith & spirituality', ' fantasy', ' foreign', ' gay & lesbian',
       ' health & wellness', ' history', ' holiday', ' horror',
       ' kids & family', ' lgbtq+', ' music', ' musical',
       ' mystery & thriller', ' nature', ' news', ' romance', ' sci-fi',
       ' short', ' sports', ' sports & fitness', ' stand-up', ' variety',
       ' war', ' western', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'history',
       'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [75]:
X.select_dtypes(include='object').columns


Index(['movieid', 'reviewText'], dtype='object')

In [76]:
X['movieid']

0                                          marvelous_pirate
1                 tony_montana_frodo_baggins_v_rocky_balboa
2         darth_vader_katniss_everdeen_sorcerer_donnie_d...
3                                        lara_croft_glimmer
4         jason_bourne_surreal_the_terminator_indiana_jones
                                ...                        
162753    the_joker_ethereal_captain_jack_sparrow_fantastic
162754    e.t._hannibal_lecter_vito_corleone_katniss_eve...
162755                       infinite_enigma_luke_skywalker
162756                    emerald_oracle_iron_man_wolverine
162757           james_t._kirk_phenomenal_zephyr_chronicles
Name: movieid, Length: 162758, dtype: object

In [77]:
X.columns

Index(['movieid', 'isFrequentReviewer', 'reviewText', 'audienceScore',
       'releaseDateStreamingDay', ' action', ' adventure', ' animation',
       ' anime', ' biography', ' comedy', ' crime', ' documentary', ' drama',
       ' faith & spirituality', ' fantasy', ' foreign', ' gay & lesbian',
       ' health & wellness', ' history', ' holiday', ' horror',
       ' kids & family', ' lgbtq+', ' music', ' musical',
       ' mystery & thriller', ' nature', ' news', ' romance', ' sci-fi',
       ' short', ' sports', ' sports & fitness', ' stand-up', ' variety',
       ' war', ' western', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'history',
       'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [78]:
# numeric_features1 = ['audienceScore']
# categorical_features1 = ['genre']

preprocessor2 = ColumnTransformer([
    ('moviesVector' , TfidfVectorizer(ngram_range = (1,2)) , 'movieid'),
    ('minMaxScaling', MinMaxScaler(), ['audienceScore']),
    ('vectorize' , TfidfVectorizer(ngram_range = (1,2)) , 'reviewText')
],remainder = 'passthrough', verbose_feature_names_out = True)
train = preprocessor2.fit_transform(X)

In [79]:
merged_Test=merged_Test.drop(columns=['releaseDateStreaming','releaseDateStreamingMonth','releaseDateStreaming','runtimeMinutes','originalLanguage','director', 'reviewerName'],axis=1)

In [80]:
merged_Test.columns

Index(['movieid', 'isFrequentReviewer', 'reviewText', 'audienceScore',
       'releaseDateStreamingDay', ' action', ' adventure', ' animation',
       ' anime', ' biography', ' comedy', ' crime', ' documentary', ' drama',
       ' faith & spirituality', ' fantasy', ' foreign', ' gay & lesbian',
       ' health & wellness', ' history', ' holiday', ' horror',
       ' kids & family', ' lgbtq+', ' music', ' musical',
       ' mystery & thriller', ' nature', ' news', ' romance', ' sci-fi',
       ' short', ' sports', ' sports & fitness', ' stand-up', ' variety',
       ' war', ' western', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'history',
       'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [81]:
test = preprocessor2.transform(merged_Test)

In [82]:
# test = pd.DataFrame.sparse.from_spmatrix(X_test_transformed2, columns = columns)

## Model 1 : Logistic Regression

In [83]:
merged_Test.columns

Index(['movieid', 'isFrequentReviewer', 'reviewText', 'audienceScore',
       'releaseDateStreamingDay', ' action', ' adventure', ' animation',
       ' anime', ' biography', ' comedy', ' crime', ' documentary', ' drama',
       ' faith & spirituality', ' fantasy', ' foreign', ' gay & lesbian',
       ' health & wellness', ' history', ' holiday', ' horror',
       ' kids & family', ' lgbtq+', ' music', ' musical',
       ' mystery & thriller', ' nature', ' news', ' romance', ' sci-fi',
       ' short', ' sports', ' sports & fitness', ' stand-up', ' variety',
       ' war', ' western', 'action', 'adventure', 'animation', 'biography',
       'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'history',
       'holiday', 'horror', 'kids & family', 'music', 'musical',
       'mystery & thriller', 'romance', 'sci-fi', 'stand-up', 'war',
       'western'],
      dtype='object')

In [84]:
y=merged_Train['sentiment']

In [85]:
from sklearn.linear_model import LogisticRegression

classifier =LogisticRegression(max_iter = 500, random_state = 28, C = 10000, solver = 'saga', tol = 0.01)
classifier.fit(train, y)

y_predict = classifier.predict(test)

In [86]:
submission_Data = pd.DataFrame({'sentiment': y_predict})
submission_Data.index.names = ['id']
submission_Data.to_csv('submission.csv')

In [87]:
df = pd.read_csv('submission.csv')

# Count the number of rows with positive and negative sentiment
positive_count = (df['sentiment'] == 'POSITIVE').sum()
negative_count = (df['sentiment'] == 'NEGATIVE').sum()

print("Number of rows with positive sentiment:", positive_count)
print("Number of rows with negative sentiment:", negative_count)

Number of rows with positive sentiment: 43154
Number of rows with negative sentiment: 12161


# Model 2 : Naive Bayes

# Model 3 : Random Forest Classifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Model 4 : Voting Classifier (Ensembling)

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier(estimators=[('lr', classifier), ('rf', random_forest)], voting='hard')

In [ ]:
# ensemble_model.fit(train, y)

In [ ]:
# y_predict = ensemble_model.predict(test)

In [ ]:
# X_test_transformed2.shape

In [ ]:
submission_Data = pd.DataFrame({'sentiment': y_predict})
submission_Data.index.names = ['id']
submission_Data.to_csv('submission.csv')